In [180]:
import pandas as pd
import numpy as np
from datetime import timedelta
from sklearn.impute import KNNImputer
pd.options.mode.chained_assignment = None

In [168]:
baseline_df = pd.read_csv("data/data_by_table/baseline.csv")
ventilator_setting_df = pd.read_csv("data/data_by_table/mimiciv_derived_ventilator_setting.csv")
vitalsign_df = pd.read_csv("data/data_by_table/mimiciv_derived_vitalsign.csv")
labevents_df = pd.read_csv("data/data_by_table/mimiciv_hosp_labevents.csv")
cohort_subject_id_stay_id_df = pd.read_csv("data/data_by_table/cohort_subject_id_stay_id.csv")
ground_truth_df = pd.read_csv("data/data_by_table/ground_truth.csv")
labevents_df['O2_flow'].replace('___', pd.NA, inplace=True)

In [32]:
def generate_df_template(stay_id, endtime):
    end_time = pd.to_datetime(endtime).floor('H')
    time_intervals = [end_time - timedelta(hours=i) for i in range(24)]
    df = pd.DataFrame(time_intervals, columns=['charttime'])
    df['stay_id'] = stay_id 
    return df

In [149]:
def generate_all_template(ground_truth_df):
    data_template = pd.DataFrame()
    for index, row in ground_truth_df.iterrows():
        data_now = generate_df_template(row['stay_id'],row['endtime'])
        data_template = pd.concat([data_template, data_now], ignore_index=False)
    return data_template

In [146]:
def fill_na(df):
    df['charttime'] = pd.to_datetime(df['charttime'])
    df = df.sort_values(by=['stay_id', 'charttime'])
    df = df.set_index('charttime')
    df_resampled = df.groupby('stay_id').resample('H').max()
    for col in df.columns:
        if(col != 'stay_id' and col != 'subject_id' and col != 'charttime'):
            df_resampled[col] = df_resampled[col].fillna(method='ffill')
            df_resampled[col] = df_resampled[col].fillna(method='bfill')
    df_resampled = df_resampled.drop(columns='stay_id')
    df_resampled = df_resampled.reset_index()
    for col in df.columns:
        if col == 'subject_id':
            df_resampled = df_resampled.drop(columns='subject_id')
        if col == 'hadm_id':
            df_resampled = df_resampled.drop(columns='hadm_id')
    return df_resampled

In [175]:
def fill_and_merge(data_all, df_target, ground_truth_df):
    template = generate_all_template(ground_truth_df)
    df_target = fill_na(df_target)
    df = pd.merge(df_target, template, how='outer', on=['stay_id','charttime'])
    df['charttime'] = pd.to_datetime(df['charttime'])
    df = df.sort_values(by=['stay_id', 'charttime'])
    df = df.set_index('charttime')
    df_resampled = df.groupby('stay_id').resample('H').max()
    for col in df.columns:
        if(col != 'stay_id' and col != 'subject_id' and col != 'charttime'):
            df_resampled[col] = df_resampled[col].fillna(method='ffill')
            df_resampled[col] = df_resampled[col].fillna(method='bfill')
    df_resampled = df_resampled.drop(columns='stay_id')
    df_resampled = df_resampled.reset_index()
    df = pd.merge(data_all,df_resampled, how='inner', on=['stay_id','charttime'])
    return df
    

In [69]:
def check_missing_values(df):
    if df.isna().any().any():
        return 1 
    else:
        return 0 
def output_id_missing(flag_data_df,data_df):
    ass_hole = []
    for index, row in flag_data_df.iterrows():
        id_df = data_df[data_df['stay_id'] == row['stay_id']]
        if check_missing_values(id_df):
            ass_hole.append(row['stay_id'])
    return ass_hole

In [194]:
def add_label_id(df,stay_id, flag, r_v, dod):
    selected_data = df[df['stay_id'] == stay_id]
    selected_data['label'] = flag
    if np.isnan(r_v):
        selected_data['Rev_h'] = -1000
    else:
         selected_data['Rev_h'] = r_v
    if np.isnan(dod):
        selected_data['dod_h'] = -1000
    else:
         selected_data['dod_h'] = dod
    selected_data = selected_data.sort_values(by=['stay_id', 'charttime'])
    return selected_data

In [192]:
def add_label(df, df_label):
    cancate_data = pd.DataFrame()
    for index, row in df_label.iterrows():
        data_now = add_label_id(df,row['stay_id'],row['label'], row['re_vent_time_diff'], row['weaning_till_dod_hr'])
        cancate_data = pd.concat([cancate_data, data_now], ignore_index=False)
    return cancate_data

### baseline_df, ventilator_setting_df, vitalsign_df, labevents_df

In [178]:
data_template = generate_all_template(ground_truth_df)
labevents_df_24 = fill_and_merge(data_template, ventilator_setting_df, ground_truth_df)
labevents_df_24 = fill_and_merge(data_template, labevents_df, ground_truth_df)
print("labevents_df finish")
vitalsign_df_24 = fill_and_merge(data_template, vitalsign_df, ground_truth_df)

labevents_df finish


In [181]:
imputer = KNNImputer(n_neighbors=2)
baseline_df_p = baseline_df 
baseline_df_p[['height_cm', 'weight_kg']] = imputer.fit_transform(baseline_df_p[['height_cm', 'weight_kg']])
baseline_df_p['height_cm'].fillna(baseline_df_p.groupby('gender')['height_cm'].transform('mean'), inplace=True)
baseline_df_p['weight_kg'].fillna(baseline_df_p.groupby('gender')['weight_kg'].transform('mean'), inplace=True)
baseline_df_p.drop(columns=['subject_id', 'hadm_id'])

,stay_id,age_now,gender,insurance,race,admission_type,first_careunit,weight_kg,height_cm,tobacco
0,35043893,23,M,Other,WHITE,EW EMER.,Trauma SICU (TSICU),87.3,180.0,1
1,32368521,37,M,Medicaid,HISPANIC OR LATINO,EW EMER.,Surgical Intensive Care Unit (SICU),115.5,188.0,0
2,31389801,40,M,Other,UNKNOWN,URGENT,Cardiac Vascular Intensive Care Unit (CVICU),63.5,175.0,1
3,30829378,41,F,Medicaid,BLACK/AFRICAN AMERICAN,URGENT,Medical Intensive Care Unit (MICU),80.2,160.0,1
4,32013423,46,F,Medicare,ASIAN,EW EMER.,Medical/Surgical Intensive Care Unit (MICU/SICU),45.9,152.0,0
...,...,...,...,...,...,...,...,...,...,...
2835,34351511,70,M,Other,WHITE,EW EMER.,Medical/Surgical Intensive Care Unit (MICU/SICU),99.5,178.0,0
2836,33181285,60,F,Medicaid,BLACK/AFRICAN AMERICAN,EW EMER.,Medical Intensive Care Unit (MICU),102.5,160.0,1
2837,34478477,63,M,Medicare,WHITE - EASTERN EUROPEAN,SURGICAL SAME DAY ADMISSION,Surgical Intensive Care Unit (SICU),87.6,165.0,0
2838,32471162,54,F,Medicare,BLACK/AFRICAN AMERICAN,OBSERVATION ADMIT,Trauma SICU (TSICU),116.0,165.0,1


In [189]:
merged_df = pd.merge(labevents_df_24, vitalsign_df_24, on=['stay_id', 'charttime'], how='inner')
merged_df = pd.merge(merged_df, labevents_df_24, on=['stay_id', 'charttime'], how='inner')
merged_df = pd.merge(merged_df, baseline_df, on=['stay_id'], how='inner')
merged_df = merged_df.drop(columns=['subject_id', 'hadm_id'])

In [195]:
final_data = add_label(merged_df, ground_truth_df)

In [191]:
ass_holes = output_id_missing(ground_truth_df,merged_df)
print(len(merged_df))
print(len(ass_holes))
print((ass_holes))

68160
0
[]


In [196]:
stay_id_to_find = 30047981
test_df = final_data
df_str = test_df[test_df['stay_id'] == stay_id_to_find].to_string(index=False)
for line in df_str.split('\n'):
    print(line)

          charttime  stay_id O2_flow_x  heart_rate   sbp  dbp  mbp  resp_rate  spo2 O2_flow_y  age_now gender insurance  race admission_type                     first_careunit  weight_kg  height_cm  tobacco  label   Rev_h   dod_h
2141-12-18 21:00:00 30047981       122        82.0  98.0 59.0 69.0       14.0  99.0       122       53      F  Medicaid WHITE         URGENT Medical Intensive Care Unit (MICU)       84.7      168.0        0      1 -1000.0 -1000.0
2141-12-18 22:00:00 30047981       122       106.0 149.0 85.0 99.0       26.0  96.0       122       53      F  Medicaid WHITE         URGENT Medical Intensive Care Unit (MICU)       84.7      168.0        0      1 -1000.0 -1000.0
2141-12-18 23:00:00 30047981       122        84.0 128.0 75.0 88.0       14.0  98.0       122       53      F  Medicaid WHITE         URGENT Medical Intensive Care Unit (MICU)       84.7      168.0        0      1 -1000.0 -1000.0
2141-12-19 00:00:00 30047981       122        87.0 143.0 80.0 94.0       16.0  9

In [197]:
final_data.to_csv("./pre_24h_data.csv")